In [1]:
import pickle
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import copy

In [2]:
np.random.seed(42)
teams = ['BKN', 'MIL', 'GSW', 'LAL', 'IND', 'CHA', 'CHI', 'DET',
          'WAS', 'TOR', 'BOS', 'NYK', 'CLE', 'MEM', 'PHI', 'NOP',
          'HOU','MIN', 'ORL', 'SAS', 'OKC', 'UTA', 'SAC', 'POR',
          'DEN', 'PHX', 'DAL', 'ATL', 'MIA', 'LAC']
teams = ['CLE', 'MEM', 'PHI', 'NOP',
          'HOU','MIN', 'ORL', 'SAS', 'OKC', 'UTA', 'SAC', 'POR',
          'DEN', 'PHX', 'DAL', 'ATL', 'MIA', 'LAC']
window = 4 # 4 samples per game (one sample / quarter)

In [3]:
N_players = 5
players = [f'player{i}' for i in range(1, N_players+1)]
n_components = 3 # num of hidden states
n_features = 3 # num of observed states
O_symbols = [0, 1, 2] # under-, avg-, over- performance
H_symbols = [0, 1, 2] # corresponding mental states
T = 100
learning_iterations = 100
initial_dist = np.array([0, 1, 0])

In [4]:
# === M ===
avg_transO = np.array([[0.5, 0.3, 0.2],
                       [0.25, 0.5, 0.25],
                       [0.2, 0.3, 0.5]])

star_transO = np.array([[0.7, 0.3, 0],
                        [0.1, 0.8, 0.1],
                        [0, 0.3, 0.7]])
# === N ===
avg_transH = np.array([[0.8, 0.2, 0.0],
                       [0.1, 0.8, 0.1],
                       [0.0, 0.2, 0.8]])

# === emission_prob ===
avg_emission = np.array([[0.7, 0.3, 0],
                       [0.1, 0.8, 0.1],
                       [0.0, 0.3, 0.7]])

# ===  R ===
R_singleH = np.array([1] + [0] * len(players))
def R_singleHO(player):
    i = int(player[-1])
    arr = [0] * (len(players)+1)
    arr[0] = 0.7
    arr[i] = 0.3
    return np.array(arr)

def R_singleO(player):
    i = int(player[-1])
    arr = [0] * (len(players)+1)
    arr[i] = 1
    return np.array(arr)

def R_star(player, star):
    if player == star:
        return R_singleH
    arr = [0] * (len(players)+1)
    i = int(star[-1])
    arr[i] = 1
    return np.array(arr)
    
R_uniform = np.array([1/(1 + len(players))] * (len(players) + 1))

In [5]:
def init_trans():
    trans_mat = np.zeros((n_components, n_components))
    
    trans_mat[0][0] = np.random.uniform(0.5, 1)
    trans_mat[0][1] = 1 - trans_mat[0][0]
    trans_mat[0][2] = 0.0
    
    trans_mat[1][1] = np.random.uniform(0.5, 1)
    trans_mat[1][0] = (1 - trans_mat[1][1]) / 2
    trans_mat[1][2] = (1 - trans_mat[1][1]) / 2
    
    trans_mat[2][2] = np.random.uniform(0.5, 1)
    trans_mat[2][0] = 0.0
    trans_mat[2][1] = 1 - trans_mat[2][2]
    
    return trans_mat

def init_random():
    mat = np.random.random((n_features, n_components))
    for i in range(n_features):
        mat[i] = mat[i] / np.sum(mat[i])
    return mat

def init_R():
    R = dict()
    for p in players:
        # populate R[p] with random numbers
        R[p] =  np.random.rand(N_players + 1)
        
        # randomly select entries and set them to 0
        nums = np.random.choice(range(N_players + 1), np.random.choice(range(N_players)), replace=False)
        for i in nums:
            R[p][i] = 0
            
        # normalize
        R[p] = R[p] / np.sum(R[p])
    
    return R

In [6]:
def cond(player, h1, h2, t, M_, N_, R_): # P(H_t^player = h1 | H_{t-1}^player = h2, O_{t-1})    
    # Requires M_, N_, R_, Os
    v = [N_[h2][h1]]
    for teammate in players:
        v.append(M_[Os[teammate][t-1]][h1])
            
    v = np.array(v)
    return np.dot(R_[player], v)  

In [7]:
def learn_R(M, N, samples, O):
    R = {p: cp.Variable(len(players) + 1, nonneg=True) for p in players}

    objective = 0
    for H in samples:
        for p in players:
            for t in range(1, T):
                h_t = H[p][t] # state of player p at time t
                v_t = [N[H[p][t-1]][h_t]]
                for teammate in players:
                    v_t.append(M[O[teammate][t-1]][h_t])
                v_t = np.array(v_t)
                prod = R[p] @ v_t
                objective -= cp.log(prod)

    R_constraints = [cp.sum(R[p]) == 1 for p in R]
    prob = cp.Problem(cp.Minimize(objective), R_constraints)
    prob.solve(solver=cp.MOSEK)
    
    R_optimized = {p: R[p].value for p in players}
    
    return R_optimized, prob.value

In [8]:
def learn_M_N(R, samples, O):
    M = cp.Variable((n_features, n_components), nonneg=True)
    N = cp.Variable((n_components, n_components), nonneg=True) 
    E = cp.Variable((n_components, n_features), nonneg=True)
    
    objective = 0
    for H in samples:
        for p in players:
            for t in range(1, T):
                h_t = H[p][t] # state of player p at time t
                v_t = [2*N[H[p][t-1], h_t]]
                for teammate in players:
                    v_t.append(M[O[teammate][t-1], h_t])
                v_t = np.array(v_t)
                prod = R[p] @ v_t
                objective -= cp.log(prod)
    
    M_constraints = [cp.sum(M[i, :]) == 1 for i in range(n_features)]
    N_constraints = [cp.sum(N[i, :]) == 1 for i in range(n_components)]
    
    constraints = M_constraints + N_constraints
    prob = cp.Problem(cp.Minimize(objective), constraints)
    prob.solve(solver=cp.MOSEK)
    
    M_optimized = M.value
    N_optimized = N.value
    
    return M_optimized, N_optimized, prob.value

In [9]:
def learn_E(samples, O):
    E = cp.Variable((n_components, n_features), nonneg=True)
    
    objective = 0
    for H in samples:
        for p in players:     
            for t in range(T):
                objective -= cp.log(E[H[p][t]][O[p][t]])
                
    E_constraints = [cp.sum(E[i, :]) == 1 for i in range(n_components)]
    prob = cp.Problem(cp.Minimize(objective), E_constraints)
    prob.solve(solver=cp.MOSEK)
    E_optimized = E.value
    
    return E_optimized, prob.value

In [10]:
def calculate_forward(M_, N_, E_, R_):
    alpha = {p: [] for p in players}
    alpha_help = {p: [] for p in players}
    
    # Initialize forward parameters
    for p in players:
        alpha_help[p].append(initial_dist)
    for p in players:
        arr = np.array([E_[h][Os[p][0]] * alpha_help[p][0][h] for h in H_symbols])
        arr /= np.sum(arr)
        alpha[p].append(arr)

    # Compute forward parameters (bottom-up)
    for p in players:
        for t in range(1, T):
            if t % 4 == 0:
                alpha_help[p].append(initial_dist)
                alpha[p].append(initial_dist)
                continue
                
            arr = [sum([cond(p, h, h_, t, M_, N_, R_) * alpha[p][t-1][h_] for h_ in H_symbols]) for h in H_symbols]
            arr = np.array(arr)
            alpha_help[p].append(arr)
            
            arr = [E_[h][Os[p][t]] * alpha_help[p][t][h] for h in H_symbols]
            arr = np.array(arr)
            arr /= np.sum(arr)
            alpha[p].append(arr)
            
    return alpha, alpha_help

In [16]:
def E_step(M_, N_, E_, R_, num_of_samples = 10):
    alpha, alpha_help = calculate_forward(M_, N_, E_, R_)
    
    # Sample hidden states from the posterior distribution
    samples = []
    for _ in range(num_of_samples):
        Hs_ = {p: [1] for p in players}

        for p in players:
            for t in range(1, T):
                dist = np.array([alpha[p][t][h] * cond(p, h, Hs_[p][t-1], t, M_, N_, R_) / alpha_help[p][t][h] if alpha_help[p][t][h] != 0 else 0 for h in H_symbols])
                dist /= np.sum(dist)
                Hs_[p].append(np.random.choice(H_symbols, p=dist))
        for p in players:
            Hs_[p] = np.array(Hs_[p])
        samples.append(copy.deepcopy(Hs_))
    
    return samples

def M_step(samples, M, N, R, E, iterations=1):
    M_opt = M
    N_opt = N
    R_opt = copy.deepcopy(R)
    E_opt = E
    val = 1e+100
    for _ in range(iterations):
        try:
#             R_opt, val1 = learn_R(M_opt, N_opt, samples, Os) # Fix M, N and maximize R, E
#             M_opt, N_opt, val1 = learn_M_N(R_opt, samples, Os) # Fix R and maximize M, N, E

            M_opt, N_opt, val1 = learn_M_N(R_opt, samples, Os) # Fix R and maximize M, N, E
            R_opt, val1 = learn_R(M_opt, N_opt, samples, Os) # Fix M, N and maximize R, E
            E_opt, val2 = learn_E(samples, Os)
            
            val = (val1 + val2) / len(samples)
        except cp.error.SolverError as e:
            print(e)
            break
            
    return M_opt, N_opt, R_opt, E_opt, val

def EM(params, iterations = 30, reltol = 1e-3, inctol = 2):
    # Initialize parameters
    M_, N_, E_, R_ = params['M'], params['N'], params['E'], params['R']
    
    cntObjIncreasing = 0
    val = 1e+100 # best objective value achieved so far
    M_opt = M_
    N_opt = N_
    E_opt = E_
    R_opt = copy.deepcopy(R_)
    for i in range(iterations):
        print(f'=== EM iteration {i+1} ===')
        # E-step
        print(f'E-step...')
        samples = E_step(M_, N_, E_, R_)
        
        
        # M-step
        print(f'M-step...')
        M_, N_, R_, E_, obj_val = M_step(samples, M_, N_, R_, E_)
        
            
        print(f'\tObjVal = {obj_val}')
        # Update current best parameters
        if obj_val < val:
            M_opt = M_
            N_opt = N_
            E_opt = E_
            R_opt = copy.deepcopy(R_)
        
        # Stopping conditions
        if abs(val - obj_val) / val < reltol and obj_val < val:
            break
        
        cntObjIncreasing = 0 if obj_val <= val else cntObjIncreasing + 1
        if cntObjIncreasing > inctol:
            break
            
        # Update best objective value so far    
        val = min(val, obj_val)
        
    return M_opt, N_opt, R_opt, E_opt, val

In [12]:
def EM_helper(param_list):
    i = 1
    val_opt = 1e+100
    for params in param_list:
        print(f'==== EM run {i} ====')
        i += 1
        M_, N_, R_, E_, val = EM(params)
        
        if val < val_opt:
            val_opt = val
            M_opt = M_
            N_opt = N_
            E_opt = E_
            R_opt = R_
            
    return M_opt, N_opt, R_opt, E_opt, val_opt

In [13]:
def print_params(M_, N_, E_, R_):
    print(f'M = {np.round(M_, 2)}')
    print(f'N = {np.round(N_, 2)}')
    print(f'E = {np.round(E_, 2)}')
    for p in players:
        print(f'R[{p}] = {np.round(R_[p], 2)}')

In [14]:
def likelihood(params): # Calculate log P(O) (given the model parameters)
    M, N, E, R = params['M'], params['N'], params['E'], params['R']

    # Caluclate forward parameters
    alpha, alpha_help = calculate_forward(M, N, E, R)
    obj = 0
    for p in players:
        for t in range(T):
            if isinstance(E, dict):
                obj += -np.log(sum([E[p][h][Os[p][t]] * alpha_help[p][t][h] for h in H_symbols])) # Pr(O_t^i | O_{1:t-1})
            else:
                obj += -np.log(sum([E[h][Os[p][t]] * alpha_help[p][t][h] for h in H_symbols])) # Pr(O_t^i | O_{1:t-1})

    return obj

def likelihood_test(params_list):
    # Given a list of model parameters returns the model parameters that achieve the highest likelihood
    val_opt = 1e+100
    for params in params_list:
        val = likelihood(params)
        if val < val_opt:
            val_opt = val
            params_opt = params
    return params_opt

In [ ]:
for team in teams:
    try:
        print(f'Optimizing team = {team}')
        with open(f'../team-data/observations/{team}_observations.pickle', 'rb') as file:
                observations = pickle.load(file)

        k = list(observations.keys())[0]
        players = list(observations[k].keys())

        Os = {p: [] for p in players}
        for game_id in observations:
            game_stats = observations[game_id]
            for p in players:
                Os[p].extend(game_stats[p])

        with open(f'./profiles2/{team}_profile.pkl', 'rb') as file:
            params = pickle.load(file)

        print(f"Initial Likelihood = {likelihood(params)}")
        print(params)
        M_, N_, R_, E_, val = EM(params, 20)
        print_params(M_, N_, E_, R_)

        print(f"Final Likelihood = {likelihood({'M': M_, 'N': N_, 'E': E_, 'R': R_})}")

        params = dict()
        params['M'] = M_
        params['N'] = N_
        params['E'] = E_
        params['R'] = R_
        with open(f'./results/{team}_profile.pkl', 'wb') as file:
            pickle.dump(params, file)
            
    except Exception as e:
        print(f'Error for team {team}')
        print(e)

Optimizing team = CLE
Initial Likelihood = 407.5198463741693
{'M': array([[0.6, 0.3, 0.1],
       [0.2, 0.6, 0.2],
       [0.1, 0.3, 0.6]]), 'N': array([[0.8, 0.2, 0. ],
       [0.1, 0.8, 0.1],
       [0. , 0.2, 0.8]]), 'E': array([[0.7, 0.2, 0.1],
       [0.1, 0.8, 0.1],
       [0.1, 0.2, 0.7]]), 'R': {'Darius Garland': array([0.5, 0. , 0.2, 0. , 0.2, 0.1]), 'Lauri Markanen': array([1, 0, 0, 0, 0, 0]), 'Isaac Okoro': array([0.5, 0. , 0.2, 0. , 0.2, 0.1]), 'Evab Mobley': array([1, 0, 0, 0, 0, 0]), 'Jarrett Allen': array([1, 0, 0, 0, 0, 0])}, 'name': 'synthetic-100'}
=== EM iteration 1 ===
E-step...
M-step...
	ObjVal = 619.2359331267337
=== EM iteration 2 ===
E-step...
M-step...
	ObjVal = 589.4324539772311
=== EM iteration 3 ===
E-step...
M-step...
	ObjVal = 550.7296221957164
=== EM iteration 4 ===
E-step...
M-step...
	ObjVal = 520.9214759995155
=== EM iteration 5 ===
E-step...
M-step...
	ObjVal = 498.85950443138853
=== EM iteration 6 ===
E-step...
M-step...
	ObjVal = 489.75713359631084

In [ ]:
players